  \begin{equation}\label{eq:ndlm}
    \dot{x}_1=\sigma x_2\,,\quad \dot{x}_2=-x_1x_3+rx_1\,,\quad \dot{x}_3=x_1x_2
  \end{equation}
  $x_4=\sigma, x_5=r$

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
def system(y, t):
    x1, x2, x3, a, b = y
    dx1_dt = a*x2
    dx2_dt = x1*(b-x3)
    dx3_dt = x1*x2
    da_dt = 0
    db_dt = 0
    return [dx1_dt, dx2_dt, dx3_dt, da_dt, db_dt]
num_trajectories = 8
t = np.linspace(0, 10, 50)
trajectory_data = []
initial_conditions = [
    [1.1695431377507957, -1.8620400264659596, 0.6747102470907413, 2.201492683925648, 1.8948235601443928],
    [-0.8237390988624025, -1.993615436372845, 0.04568446215026378, 0.8617312492505408, 1.101770774346663],
    [0.12957073531781393, 0.4729183623172455, 0.2929256922445028, 1.467438322595938, 1.2776176089750717],
    [1.6904165220848717, 0.004596973297678275, 0.7954861806097131, 2.5971545910899403, 0.6873830363529853],
    [-0.0015205077281947865, 0.5879868502684094, 0.8791007454853291, 3.1422769343530974, 0.375581166866535],
    [0.7319757309185726, -0.48369679551335754, 0.8061013443928233, 2.168023257083404, 0.5790403391356412],
    [-1.4871777853005659, 1.8045190306281786, 0.15339323614150724, 0.38327528737059147, 0.6415053089060836],
    [1.9182568720920186, -0.7335119211838745, 0.662765164899644, 3.553361820286609, 0.8373018598409908]
]
for r in range(num_trajectories):
    initial_condition = initial_conditions[r]
    sol = odeint(system, initial_condition, t)
    trajectory_data.append(sol)
with open('50.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['x1', 'x2', 'x3', 'a', 'b', 'trajectory']) 
    for r, data in enumerate(trajectory_data):
        for j in range(len(t)):
            x1 = data[j, 0]
            x2 = data[j, 1]
            x3 = data[j, 2]
            a = data[j, 3]
            b = data[j, 4]
            writer.writerow([x1, x2, x3, a, b, r+1])
            
# for r, data in enumerate(trajectory_data):
#     plt.plot(t, data[:, 0], label=f'Trajectory {r + 1}')

# plt.xlabel('Time')
# plt.ylabel('x1')
# plt.title('Trajectories of x1')
# plt.legend()
# plt.show()             

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'C:\Users\mebratie\Desktop\KR\KR_Regression\Data\Lorenz_System\Lorenz_System_T8_50_1'
def generate_random_values():
    x1 = random.uniform(0.5, 4)
    x2 = random.uniform(0.3, 4)
    x3 = random.uniform(0.2, 4)
    x4 = random.uniform(0.4, 4)
    x5 = random.uniform(0.4, 4)
    return x1, x2, x3, x4, x5
def generate_random_values_based_on_c():
    x1 = random.uniform(0.5, 4)
    x2 = random.uniform(0.3, 4)
    x3 = random.uniform(0.2, 4)
    x4 = random.uniform(0.4, 4)
    x5 = random.uniform(0.4, 4)
    return x1, x2, x3, x4, x5
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4 = y
        f = np.array([ x4*x2, x1*(x5-x3), x1*x2, 0, 0])
        normalized_f = normalize(f)
        return normalized_f
    num_trajectories = 8
    t = np.linspace(0, 10, 50) # 10 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4, x5):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]}, {initial_condition[4]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    num_variables = 5 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row)  
    output_directory1 = r'C:\Users\mebratie\Desktop\KR\KR_Regression\results\Lorenz_System\Lorenz_System_T8_50_1'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('C:\\Users\\mebratie\\Desktop\\KR\\KR_Regression\\Data\\Lorenz_System\\Lorenz_System_T8_50_1\\trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4, x5 = generate_random_values()
    initial_conditions = [generate_random_values_based_on_c() for _ in range(8)]  # number of trajectories
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 9):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

In [30]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff

output_directory = r'C:\Users\mebratie\Desktop\KR\KR_Regression\Data\Lorenz_System\Lorenz_System_T8_50_1'

def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    
    def normalized_system(y, t):
        x1, x2, x3, x4, x5 = y
        f = np.array([x4*x2, x1*(x5-x3), x1*x2, 0, 0])
        normalized_f = normalize(f)
        return normalized_f
    
    num_trajectories = 8
    t = np.linspace(0, 10, 50) # 10 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4, x5):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]}, {initial_condition[4]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    
    num_variables = 5 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row)  
    
    output_directory1 = r'C:\Users\mebratie\Desktop\KR\KR_Regression\results\Lorenz_System\Lorenz_System_T8_50_1'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()

def split_data():
    trajectories = {}
    column_names = None
    with open('C:\\Users\\mebratie\\Desktop\\KR\\KR_Regression\\Data\\Lorenz_System\\Lorenz_System_T8_50_1\\trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)

if __name__ == "__main__":
    initial_conditions = [
        [1.1695431377507957, -1.8620400264659596, 0.6747102470907413, 2.201492683925648, 1.8948235601443928],
        [-0.8237390988624025, -1.993615436372845, 0.04568446215026378, 0.8617312492505408, 1.101770774346663],
        [0.12957073531781393, 0.4729183623172455, 0.2929256922445028, 1.467438322595938, 1.2776176089750717],
        [1.6904165220848717, 0.004596973297678275, 0.7954861806097131, 2.5971545910899403, 0.6873830363529853],
        [-0.0015205077281947865, 0.5879868502684094, 0.8791007454853291, 3.1422769343530974, 0.375581166866535],
        [0.7319757309185726, -0.48369679551335754, 0.8061013443928233, 2.168023257083404, 0.5790403391356412],
        [-1.4871777853005659, 1.8045190306281786, 0.15339323614150724, 0.38327528737059147, 0.6415053089060836],
        [1.9182568720920186, -0.7335119211838745, 0.662765164899644, 3.553361820286609, 0.8373018598409908]
    ]
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 9):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4, x5):
(1.1695431377507957, -1.8620400264659596, 0.6747102470907413, 2.201492683925648, 1.8948235601443928)
(-0.8237390988624025, -1.993615436372845, 0.04568446215026378, 0.8617312492505408, 1.101770774346663)
(0.12957073531781393, 0.4729183623172455, 0.2929256922445028, 1.467438322595938, 1.2776176089750717)
(1.6904165220848717, 0.004596973297678275, 0.7954861806097131, 2.5971545910899403, 0.6873830363529853)
(-0.0015205077281947865, 0.5879868502684094, 0.8791007454853291, 3.1422769343530974, 0.375581166866535)
(0.7319757309185726, -0.48369679551335754, 0.8061013443928233, 2.168023257083404, 0.5790403391356412)
(-1.4871777853005659, 1.8045190306281786, 0.15339323614150724, 0.38327528737059147, 0.6415053089060836)
(1.9182568720920186, -0.7335119211838745, 0.662765164899644, 3.553361820286609, 0.8373018598409908)
